## Historical and forecast data electricity demand of Germany 15-min

In [ ]:
# packages
import os
import pandas as pd
from datetime import datetime

In [ ]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Electricity demand data of Germany**

Website: https://www.smard.de/home/downloadcenter/download-marktdaten/?downloadAttributes=%7B%22selectedCategory%22:2,%22selectedSubCategory%22:5,%22selectedRegion%22:%22DE%22,%22selectedFileType%22:%22CSV%22,%22from%22:1577833200000,%22to%22:1704149999999%7D

In [ ]:
df_el_demand = pd.read_csv("/content/drive/My Drive/ms_wind_curtailment_prediction/de_electricity_demand_2020_24.csv", sep = ';')

columns_to_keep = ['Datum', 'Anfang', 'Ende', 'Gesamt (Netzlast) [MWh] Originalauflösungen',
                  'Residuallast [MWh] Originalauflösungen',
                   'Pumpspeicher [MWh] Originalauflösungen']

column_name_mapping = {
    'Datum': 'date',
    'Anfang': 'start',
    'Ende' : 'end',
    'Gesamt (Netzlast) [MWh] Originalauflösungen': 'total_grid_load_MWh',
    'Residuallast [MWh] Originalauflösungen': 'residual_load_MWh',
    'Pumpspeicher [MWh] Originalauflösungen': 'pumped_storage_MWh'
}

df_el_demand = df_el_demand[columns_to_keep].rename(columns=column_name_mapping)

# Convert time columns to datetime
df_el_demand['start'] = pd.to_datetime(df_el_demand['date'] + ' ' + df_el_demand['start'])
df_el_demand = df_el_demand.drop('date', axis = 1)
df_el_demand = df_el_demand.drop('end', axis = 1)
df_el_demand.set_index('start', inplace=True)

columns_to_float = ['total_grid_load_MWh', 'residual_load_MWh', 'pumped_storage_MWh']

for column in columns_to_float:
  df_el_demand[column] = df_el_demand[column].str.replace('.', '')
  df_el_demand[column] = df_el_demand[column].str.replace(',', '.')
  df_el_demand[column] = df_el_demand[column].astype(float)

df_el_demand = df_el_demand.div(15).round(2)

<ipython-input-99-d5f098cca720>:28: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_el_demand[column] = df_el_demand[column].str.replace('.', '')


In [ ]:
# save csv
df_el_demand.to_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/electricity_demand_2020_24.csv',sep = ';', index_label='timestamp')